In [1]:
# Import Libraries
import sys
import math
import glob, os
import cv2
import numpy as np
import pandas as pd
import geopy.distance
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.measure
from skimage import exposure
from skimage import feature


2024-07-11 17:36:58.605394: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 17:36:59.578504: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2024-07-11 17:36:59.578636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/sit

In [2]:
# Setup paths and import local libraries
%cd ~/Projects/
%cd SkyAI-Sim

prj_path = Path('SkyAI-Sim')
sys.path.insert(0, str(prj_path))
from src.utils import geo_helper, io_helper, img_helper
from src.utils.io_helper import pretty, import_module
from src.utils import consts
from src.data.googlemap import GoogleMap

args = consts.ARGS
args

/home/sdjkhosh/Projects
/home/sdjkhosh/Projects/SkyAI-Sim
Namespace(coords='dataset/Test/coords_list.txt', fov=78.8, aspect_ratio=[4, 3], overlap=0, utm='EPSG:32616', data_dir='dataset/Test', img_size=[400, 400, 3], batch_size=8, seed=2024)


usage: SkyAI Sim Project [-h] [--coords COORDS] [--fov FOV]
                         [--aspect_ratio ASPECT_RATIO [ASPECT_RATIO ...]]
                         [--overlap OVERLAP] [--utm UTM] [--data_dir DATA_DIR]
                         [--img_size IMG_SIZE [IMG_SIZE ...]]
                         [--batch_size {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127}]
                         [--seed SEED]
SkyAI Sim Project: error: argument --fov: invalid float value: '/home/sdjkhosh/.local/share/jupyter/runtime/kernel-v2-1439592zX3t12C2sE52.json'


Namespace(coords='dataset/Test/coords_list.txt', fov=78.8, aspect_ratio=[4, 3], overlap=0, utm='EPSG:32616', data_dir='dataset/Test', img_size=[400, 400, 3], batch_size=8, seed=2024)

In [3]:
coords_list = np.asanyarray(pd.read_csv(args.coords, dtype=float, sep=' '))
coords_list

array([[ 4.07127760e+01, -7.40059740e+01,  5.00000000e+01],
       [ 5.15073510e+01, -1.27758000e-01,  2.00000000e+02],
       [ 3.56894870e+01,  1.39691711e+02,  1.50000000e+02],
       [ 4.88566130e+01,  2.35222200e+00,  7.50000000e+01],
       [ 5.57558250e+01,  3.76172980e+01,  1.20000000e+02],
       [-3.38688200e+01,  1.51209290e+02,  1.80000000e+02],
       [-2.35505200e+01, -4.66333080e+01,  9.00000000e+01],
       [ 3.77749290e+01, -1.22419418e+02,  1.30000000e+02],
       [ 1.35208300e+00,  1.03819839e+02,  1.10000000e+02]])

In [4]:
for coords in coords_list:
        print(coords)
        bbox_m = geo_helper.get_map_dim_m(
                args.fov, coords[-1],
                args.aspect_ratio[0]/args.aspect_ratio[1]
                )
        bbox = geo_helper.calc_bbox_m(coords[:2],
                                        bbox_m)
        args.coords = tuple(np.array(bbox).flatten()) + (coords[-1],)
        aerial_data = GoogleMap(
        args=args,
        map_type='satellite',
        data_dir=args.data_dir,
        overlap=args.overlap
        )

[ 40.712776 -74.005974  50.      ]


2024-07-11 17:37:01.932685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 17:37:02.688321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 486 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:81:00.0, compute capability: 7.5


Checking folder:
	 dataset/Test/satellite_0 Folder Exists.
8.42788501458358e-05 6.345389342499175e-05
                   TL         BR Center TL (UTM) BR (UTM) Map Size (m)   
y/lat       40.712844  40.712708    NaN      NaN      NaN          NaN  \
x/lon      -74.006093 -74.005855    NaN      NaN      NaN          NaN   
z/agl             NaN        NaN    NaN      NaN      NaN          NaN   
total/area        NaN        NaN    NaN      NaN      NaN          NaN   

           Map Size (pixel/zoom) # Raster Images 1 Image Size (m)   
y/lat                        353             NaN            20.03  \
x/lon                        266             NaN            15.02   
z/agl                         21             NaN            15.24   
total/area                   NaN             NaN           300.85   

           1 Image Size (pixel/zoom)  
y/lat                            NaN  
x/lon                            NaN  
z/agl                            NaN  
total/area               

ValueError: math domain error